In [5]:
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 11.1 MB/s eta 0:00:00


In [37]:
import pandas as pd
import numpy as np
import requests
import json
import time

In [ ]:
from IPython.display import HTML

def side_by_side(objects):
  """Plot objects side by side"""
  html = '<div style="display:flex">'
  for obj in objects:
    if isinstance(obj, pd.DataFrame):
      obj_html = obj.to_html()
    elif isinstance(obj, str):
      obj_html = obj
    html += '<div style="margin-right: 2em">'
    html += obj_html
    html += '</div>'
  html += '</div>'
  display(HTML(html))

def check_df_unique_vals(df, idx='auto_id', limit=10):
  """Use this method to display side by side
  dataframes of the unique values in the dataframe with counts"""
  if idx == '':
    print("Please provide and index column")
    return
  columns = list(df.columns)
  columns.remove(idx)
  df_list = []
  for col in columns:
    group_df = pd.DataFrame(df[[col, idx]].groupby(col, dropna=False).count())
    group_df = group_df.reset_index()
    group_df.columns = [col, 'count']
    group_df = group_df.sort_values('count', ascending=False).head(limit)
    df_list.append(group_df)
  side_by_side(df_list)

def check_df_missing_vals(df, idx='subject_id', limit=10):
  """
  Use this method to display side by side dataframes of the
  missing and non missing values in each column
  """
  df = df.copy()
  columns = list(df.columns)
  columns.remove(idx)
  df_list = []
  for col in columns:
    df[col] = df[col].replace('', np.nan)
    nan_count = df[col].isna().sum()
    non_nan_count = df[col].isna().count() - nan_count
    nan_df = pd.DataFrame({col:['NaN', 'non-NaN'], 'count':[nan_count, non_nan_count]})
    df_list.append(nan_df)
  side_by_side(df_list)

def convert_cols_to_numeric(df, exclude_cols):
  """Converts columns to numeric. If there is an error
  the value is NaN"""
  df_numeric = df.copy()
  cols = [x for x in list(df.columns) if x not in exclude_cols]
  for col in cols:
    df_numeric[col] = pd.to_numeric(df[col], errors='coerce')
  return df_numeric


In [7]:
odi_df = pd.read_csv('../data/NormativeODI_DATA_2024-01-04_1611.csv')

In [18]:
check_df_unique_vals(odi_df, idx='record_id')

,redcap_survey_identifier,count
0,NaN,218
,assessment_of_back_pain_in_people_who_never_had_sp_timestamp,count
213,[not completed],2
15,2023-11-20 21:06:56,2
17,2023-11-20 21:07:19,2
43,2023-11-20 21:12:47,2
138,2023-11-20 22:35:02,1
139,2023-11-20 22:36:04,1
140,2023-11-20 22:38:37,1
141,2023-11-20 22:41:47,1


In [10]:
age = odi_df.age
# Define conditions
conditions = [
    age < 20,
    (20 <= age) & (age < 30),
    (30 <= age) & (age < 40),
    (40 <= age) & (age < 50),
    (50 <= age) & (age < 60),
    (60 <= age) & (age < 70),
    (70 <= age) & (age < 80),
    (80 <= age)
]

# Define corresponding values for each condition
result_values = [
    "<20",
    "20-30",
    "30-40",
    "40-50",
    "50-60",
    "60-70",
    "70-80",
    "80+"
]

# Apply conditions using numpy.select
odi_df['age_range'] = np.select(conditions, result_values, default=0)

In [11]:
odi_questions = [
    'pain_intensity',
    'personal_care_e_g_washing', 'lifting', 'walking', 'sitting',
    'standing', 'sleeping', 'social_life', 'travelling',
    'employment_homemaking'
]

In [12]:
odi_clean_df = odi_df[np.all(odi_df[odi_questions] >= 1, axis=1) & np.all(odi_df[odi_questions] <= 5, axis=1)].copy()
odi_clean_df['odi_score'] = np.sum(odi_clean_df[odi_questions], axis=1)
odi_clean_df


,record_id,redcap_survey_identifier,assessment_of_back_pain_in_people_who_never_had_sp_timestamp,age,sex,height,weight_in_pounds,zip_code,how_physically_demanding_i,have_you_ever_experienced,...,employment_homemaking,attention_check___1,attention_check___2,attention_check___3,attention_check___4,attention_check___5,attention_check___6,assessment_of_back_pain_in_people_who_never_had_sp_complete,age_range,odi_score
0,1,NaN,2023-11-14 21:52:17,36,1,20,17,46221.0,5,1,...,1.0,0,0,0,1,0,1,2,30-40,14.0
1,2,NaN,2023-11-20 21:03:17,45,2,20,89,44107.0,2,1,...,2.0,1,1,0,1,1,0,2,40-50,20.0
2,3,NaN,2023-11-20 21:03:27,29,2,14,9,12508.0,3,1,...,2.0,1,1,1,1,1,1,2,20-30,33.0
3,4,NaN,2023-11-20 21:04:07,69,2,23,46,59474.0,3,1,...,1.0,1,0,0,1,1,1,2,60-70,12.0
4,5,NaN,2023-11-20 21:04:31,26,1,21,74,60647.0,1,1,...,1.0,1,1,1,1,0,0,2,20-30,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,213,NaN,2023-11-21 02:23:16,32,2,18,50,60020.0,3,1,...,3.0,0,1,1,0,1,1,2,30-40,34.0
213,214,NaN,2023-11-21 02:26:58,34,1,23,76,92672.0,3,1,...,1.0,1,1,1,1,0,0,2,30-40,13.0
214,215,NaN,2023-11-21 02:27:59,42,1,26,151,97211.0,3,1,...,2.0,1,1,1,1,0,0,2,40-50,20.0
215,216,NaN,2023-11-21 02:35:15,50,1,25,151,77001.0,2,1,...,2.0,1,1,0,1,0,1,2,50-60,23.0


In [48]:
from tableone import TableOne

## Create an instance of TableOne with the input arguments
# mytable = TableOne(table_df, columns=columns, categorical=categorical, nonnormal=nonnormal, pval=False, pval_adjust="fdr_by", decimals=1)

columns = [
      #  'assessment_of_back_pain_in_people_who_never_had_sp_timestamp', 
       'age',
       'sex', 
       'height', 
       'weight_in_pounds', 
    #    'zip_code',
      #  'how_physically_demanding_i', 
      #  'have_you_ever_experienced',
      #  'how_have_you_addressed_add', 
      #  'please_specify', 
      #  'pain_intensity',
      #  'personal_care_e_g_washing', 
      #  'lifting', 
      #  'walking', 
      #  'sitting',
      #  'standing', 
      #  'sleeping', 
      #  'social_life', 
      #  'travelling',
      #  'employment_homemaking', 
      #  'attention_check___1', 
      #  'attention_check___2',
      #  'attention_check___3', 
      #  'attention_check___4', 
      #  'attention_check___5',
      #  'attention_check___6',
      #  'assessment_of_back_pain_in_people_who_never_had_sp_complete',
       'age_range', 
       'odi_score']
nonnormal = ['age', 'height', 'weight_in_pounds', 'odi_score']
categorical = ['age_range']
mytable = TableOne(odi_clean_df, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby = ["age_range"])

## Display the table using the tabulate method. The tablefmt argument allows the table to be displayed in multiple formats, including “github”, “grid”, “fancy_grid”, “rst”, “html”, and “latex”.:
print(mytable.tabulate(tablefmt="github"))

|                                  |       | Missing   | Overall           | 20-30            | 30-40             | 40-50            | 50-60             | 60-70            | 70-80            | 80+                 |
|----------------------------------|-------|-----------|-------------------|------------------|-------------------|------------------|-------------------|------------------|------------------|---------------------|
| n                                |       |           | 205               | 28               | 70                | 50               | 26                | 24               | 6                | 1                   |
| age, median [Q1,Q3]              |       | 0         | 40.0 [34.0,50.0]  | 28.0 [25.8,28.0] | 35.0 [33.0,36.0]  | 43.5 [41.0,45.8] | 53.0 [51.0,54.0]  | 63.5 [61.0,68.0] | 71.5 [70.2,72.0] | 80.0 [80.0,80.0]    |
| sex, mean (SD)                   |       | 0         | 1.6 (0.5)         | 1.7 (0.5)        | 1.5 (0.5)         | 1.7 (0.5)        | 1.5 (

In [29]:
# Clean up the zipcode
odi_df['zip_code'] = odi_df['zip_code'].astype(str).apply(lambda code: code[:-2])

In [42]:
# Extracting the state from the zipcodes
headers = { 
  "apikey": "952f36a0-b0e6-11ee-bf04-7fb35e5ffc15"}

pause_duration = 0.5
all_results = []
for code in odi_df['zip_code']:
  params = (
    ("codes",f"{code}"),
    ("country", "US")
  );

  response = requests.get('https://app.zipcodebase.com/api/v1/search', headers=headers, params=params).json()
  if not isinstance(response['results'], list):
    all_results.append(response['results'][code])
  else:
    print(f"Failed to extract zipcode for {code}")


  # Throttling the api
  time.sleep(0.01)

Failed to extract zipcode for 3063
Failed to extract zipcode for 8260
Failed to extract zipcode for 7702
Failed to extract zipcode for n
Failed to extract zipcode for 7109
Failed to extract zipcode for 2150
Failed to extract zipcode for 2138
Failed to extract zipcode for 1501
Failed to extract zipcode for 2176


In [56]:
final_results = [result[0] for result in all_results]
locations_df = pd.DataFrame(final_results)
# reduce to only distinct rows
locations_df = locations_df.drop_duplicates().reset_index().drop(columns='index')

In [57]:
locations_df.head()

,postal_code,country_code,latitude,longitude,city,state,city_en,state_en,state_code,province,province_code
0,46221,US,39.75090000,-86.19240000,Indianapolis,Indiana,Indianapolis,Indiana,IN,Marion,097
1,44107,US,41.48470000,-81.80180000,Lakewood,Ohio,Lakewood,Ohio,OH,Cuyahoga,035
2,12508,US,41.50970000,-73.96340000,Beacon,New York,Beacon,New York,NY,Dutchess,027
3,59474,US,48.50370000,-111.83910000,Shelby,Montana,Shelby,Montana,MT,Toole,101
4,60647,US,41.92090000,-87.70430000,Chicago,Illinois,Chicago,Illinois,IL,Cook,031


In [61]:
# joining dataframes
relevant_cols = ['postal_code', 'city', 'state', 'state_code', 'latitude', 'longitude']
odi_loc_df = odi_df.merge(locations_df[relevant_cols], how='left', left_on='zip_code', right_on='postal_code')

In [62]:
odi_loc_df.head()

,record_id,redcap_survey_identifier,assessment_of_back_pain_in_people_who_never_had_sp_timestamp,age,sex,height,weight_in_pounds,zip_code,how_physically_demanding_i,have_you_ever_experienced,...,attention_check___5,attention_check___6,assessment_of_back_pain_in_people_who_never_had_sp_complete,age_range,postal_code,city,state,state_code,latitude,longitude
0,1,NaN,2023-11-14 21:52:17,36,1,20,17,46221,5,1,...,0,1,2,30-40,46221,Indianapolis,Indiana,IN,39.75090000,-86.19240000
1,2,NaN,2023-11-20 21:03:17,45,2,20,89,44107,2,1,...,1,0,2,40-50,44107,Lakewood,Ohio,OH,41.48470000,-81.80180000
2,3,NaN,2023-11-20 21:03:27,29,2,14,9,12508,3,1,...,1,1,2,20-30,12508,Beacon,New York,NY,41.50970000,-73.96340000
3,4,NaN,2023-11-20 21:04:07,69,2,23,46,59474,3,1,...,1,1,2,60-70,59474,Shelby,Montana,MT,48.50370000,-111.83910000
4,5,NaN,2023-11-20 21:04:31,26,1,21,74,60647,1,1,...,0,0,2,20-30,60647,Chicago,Illinois,IL,41.92090000,-87.70430000
